# A guide to some of the rowmat_utils functions

Using some of the ```rowmat_utils``` functions to get reacclimated, and gearing up to do some additional programming.

In [1]:
quietly do rowmat_utils_mata.do


file c:\ado\personal\lrowmat_utils.mlib could not be opened
(2 lines skipped)


r(603);
r(603);


In spite of the above error being thrown, we still have a host of quantities defined in mata:

In [2]:
mata: mata describe


      # bytes   type                        name and extent
-------------------------------------------------------------------------------
          440   real matrix                 rm_abscolsums()
          364   real matrix                 rm_absrowsums()
          224   real matrix                 rm_alpha0()
          660   real matrix                 rm_matmult()
          416   real matrix                 rm_matvecmult()
          800   real matrix                 rm_newtinv()
          364   real matrix                 rm_transpose()
          100   real matrix                 rm_vecvecmult()
-------------------------------------------------------------------------------


Quick refresher course in how to use these functions. First, we do the following:

In [3]:
mata: X = 1, 2, 3 \ 4, 5, 6 \ 7, 8, 9
mata: Y = 9, 8, 7 \ 7, 6, 4 \ 3, 2, 1
mata: X*Y




         1     2     3
    +-------------------+
  1 |   32    26    18  |
  2 |   89    74    54  |
  3 |  146   122    90  |
    +-------------------+


In [4]:
mata: rm_matmult(rowshape(X, 1), rowshape(Y, 1))

         1     2     3     4     5     6     7     8     9
    +-------------------------------------------------------+
  1 |   32    26    18    89    74    54   146   122    90  |
    +-------------------------------------------------------+


No problems there. Now, let's think about a method for creating the shulz iteration, but we should do this in such a way that it conforms with the conventions developed for the Newton iterations...

## Schulz iteration

To get a Matrix square root, we have the following algorithm (which avoids inversion and leverages routines as it can be done wholly with basic matrix operations:

$$
Y_0 = A, \quad Z_0=I
$$

$$
Y_{k+1} = \frac{1}{2}\left(3I-Z_kY_k\right)
$$
$$
Z_{k+1} = \frac{1}{2}\left(3I - Z_kY_k\right)Z_k
$$


In [5]:
mata
    real matrix rm_sqrt(real matrix A, real scalar tol, real scalar its)
    {
        real matrix ZP, YP, Z, Y, K, Id
        real scalar i
        
        i=1
        
        Z = rowshape(I(sqrt(cols(A))), 1)
        Z = Z#J(rows(A), 1, 1)
        Id = Z#J(rows(A), 1, 1)
        
        Y = A
        
        do {
            K  = (3:*Id :- rm_matmult(Z, Y))
            YP = .5:*rm_matmult(Y, K)
            ZP = .5:*rm_matmult(K, Z)
            Y = YP
            Z = ZP
            i = i + 1    
            } while ( i <its  )
        
        return(Y)
    }
end


------------------------------------------------- mata (type end to exit) ------

>     {
>         real matrix ZP, YP, Z, Y, K, Id
>         real scalar i
>         i=1
>         Z = rowshape(I(sqrt(cols(A))), 1)
>         Z = Z#J(rows(A), 1, 1)
>         Id = Z#J(rows(A), 1, 1)
>         Y = A
>         do {
>             K  = (3:*Id :- rm_matmult(Z, Y))
>             YP = .5:*rm_matmult(Y, K)
>             ZP = .5:*rm_matmult(K, Z)
>             Y = YP
>             Z = ZP
>             i = i + 1    
>             } while ( i <its  )
>         return(Y)
>     }
note: argument tol unused

: end
--------------------------------------------------------------------------------


In [6]:
mata: X = 1,2,.5 \ 0, 1.5, .4 \ .1, .1, 1.1
mata: X



         1     2     3
    +-------------------+
  1 |    1     2    .5  |
  2 |    0   1.5    .4  |
  3 |   .1    .1   1.1  |
    +-------------------+


In [9]:
mata: XX = rowshape(X, 1)
mata: XX = XX#J(4,1,1)
mata: XX[4,] = XX[4,] :+ 1
mata: XX





         1     2     3     4     5     6     7     8     9
    +-------------------------------------------------------+
  1 |    1     2    .5     0   1.5    .4    .1    .1   1.1  |
  2 |    1     2    .5     0   1.5    .4    .1    .1   1.1  |
  3 |    1     2    .5     0   1.5    .4    .1    .1   1.1  |
  4 |    2     3   1.5     1   2.5   1.4   1.1   1.1   2.1  |
    +-------------------------------------------------------+


In [15]:
mata: rm_sqrt(XX[1::3,], .01, 20)

               rm_sqrt():  3200  conformability error
                 <istmt>:     -  function returned error


r(3200);


In [16]:
mata: XX[1::3, ]

         1     2     3     4     5     6     7     8     9
    +-------------------------------------------------------+
  1 |    1     2    .5     0   1.5    .4    .1    .1   1.1  |
  2 |    1     2    .5     0   1.5    .4    .1    .1   1.1  |
  3 |    1     2    .5     0   1.5    .4    .1    .1   1.1  |
    +-------------------------------------------------------+
